# **Deformable Convolution for Lung Cancer Prediction Based on Transcriptomic Data : a Basic Convolutional Neural Network**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement a basic MLP for our binary classification task. First we pre train our MLP on the non Lung cancer dataset. Then, we fine tune it on the lung cancer dataset (we don't keep the final classification layer) :

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import datetime

from utils import *
from train import *
from data_utils import *
from modules.MLP import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

In [2]:
# device to use, if cuda available then use cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working on : ", device)

Working on :  cuda


In [3]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/mlp.yaml', 'r') as stream:
    basic_cnn_args  = yaml.load(stream,Loader=yaml.Loader)

## **1 - Pre-Training on the Non Lung Dataset :**

### **1 - 1 - Get the Data :**

In [4]:
%%time
# Getting the data
# dataset
non_lung_dataset = TranscriptomicVectorsDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = basic_cnn_args["batch_size_pt"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_pt"])

CPU times: user 12.5 s, sys: 299 ms, total: 12.8 s
Wall time: 12.3 s


### **1 - 2 - Network, Criterion and Training :**

In [5]:
# network
net = MLP(len(non_lung_dataset[0][0])).to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**basic_cnn_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_checkpt.pt')
# Tensorboard summary writer
if basic_cnn_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
    
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, non_lung_dataloader_train, non_lung_dataloader_validation, basic_cnn_args['epochs'], writer=writer)
if not basic_cnn_args['no_tensorboard']:
    writer.close()

Epoch 1/50:   5%|▍         | 2/41 [00:00<00:02, 14.38it/s, loss=1.9257e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 1/50: 100%|██████████| 41/41 [00:02<00:00, 16.58it/s, loss=1.9883e-01]


Epoch 1/50, Train Loss: 9.9510e-02, Test Loss: 0.1415
Epoch 1/50, Train Accuracy: 72.11%, Test Accuracy: 64.69%
Epoch 1/50, Train AUC: 72.06%, Test AUC: 68.94%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.95      0.63      0.76      2248
      Cancer       0.21      0.75      0.33       295

    accuracy                           0.65      2543
   macro avg       0.58      0.69      0.54      2543
weighted avg       0.86      0.65      0.71      2543



Epoch 2/50: 100%|██████████| 41/41 [00:02<00:00, 16.46it/s, loss=1.1185e-01]


...

Epoch 17/50: 100%|██████████| 41/41 [00:02<00:00, 16.40it/s, loss=4.0828e-05]


Epoch 17/50, Train Loss: 3.2555e-04, Test Loss: 0.1317
Epoch 17/50, Train Accuracy: 98.13%, Test Accuracy: 81.85%
Epoch 17/50, Train AUC: 98.11%, Test AUC: 63.51%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.87      0.89      2248
      Cancer       0.29      0.40      0.34       295

    accuracy                           0.82      2543
   macro avg       0.60      0.64      0.62      2543
weighted avg       0.84      0.82      0.83      2543



Epoch 18/50: 100%|██████████| 41/41 [00:02<00:00, 16.48it/s, loss=1.3286e-03]


Epoch 18/50, Train Loss: 1.9473e-04, Test Loss: 0.1294
Epoch 18/50, Train Accuracy: 98.78%, Test Accuracy: 83.90%
Epoch 18/50, Train AUC: 98.76%, Test AUC: 62.46%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.90      0.91      2248
      Cancer       0.32      0.35      0.33       295

    accuracy                           0.84      2543
   macro avg       0.62      0.62      0.62      2543
weighted avg       0.84      0.84      0.84      2543



KeyboardInterrupt: 

## **2 - Fine-Tuning on the Lung dataset :**

### **2 - 1 - Load Pre-Trained Model :**

In [6]:
# Load the pretrained Model
net =  MLP(len(non_lung_dataset[0][0])).to(device).double()
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

### **2 - 2 - Get the Data :**

In [7]:
%%time
# Getting the data
# dataset
lung_dataset = TranscriptomicVectorsDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = basic_cnn_args["batch_size_ft"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_ft"])

CPU times: user 11.3 s, sys: 247 ms, total: 11.5 s
Wall time: 11.5 s


### **2 - 3 - Fine Tuning Procedure :**

In [8]:
# Beginnig Of Transfer Learnig Procedure
net = fine_tune_mlp(pretrained)
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
savepath = Path('models_finetuned/fine_tuned_mlp_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, basic_cnn_args['epochs'])

Epoch 1/50:  40%|████      | 2/5 [00:00<00:00, 14.21it/s, loss=9.4688e-02]

Training on GPU 



Epoch 1/50: 100%|██████████| 5/5 [00:00<00:00, 15.87it/s, loss=9.4636e-02]


Epoch 1/50, Train Loss: 7.2001e-02, Test Loss: 0.1499
Epoch 1/50, Train Accuracy: 79.14%, Test Accuracy: 69.93%
Epoch 1/50, Train AUC: 79.53%, Test AUC: 53.97%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.71      0.81       263
      Cancer       0.08      0.37      0.14        19

    accuracy                           0.69       282
   macro avg       0.51      0.54      0.47       282
weighted avg       0.88      0.69      0.76       282



Epoch 2/50: 100%|██████████| 5/5 [00:00<00:00, 16.37it/s, loss=1.0485e-01]


...

Epoch 26/50: 100%|██████████| 5/5 [00:00<00:00, 15.73it/s, loss=7.5447e-05]


Epoch 26/50, Train Loss: 7.2088e-05, Test Loss: 0.0634
Epoch 26/50, Train Accuracy: 99.84%, Test Accuracy: 89.64%
Epoch 26/50, Train AUC: 99.82%, Test AUC: 53.17%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.96      0.95       263
      Cancer       0.15      0.11      0.12        19

    accuracy                           0.90       282
   macro avg       0.55      0.53      0.54       282
weighted avg       0.88      0.90      0.89       282



Epoch 27/50: 100%|██████████| 5/5 [00:00<00:00, 15.80it/s, loss=1.3584e-04]


Epoch 27/50, Train Loss: 9.0421e-05, Test Loss: 0.0599
Epoch 27/50, Train Accuracy: 99.69%, Test Accuracy: 89.12%
Epoch 27/50, Train AUC: 99.64%, Test AUC: 52.79%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.95      0.94       263
      Cancer       0.13      0.11      0.12        19

    accuracy                           0.89       282
   macro avg       0.53      0.53      0.53       282
weighted avg       0.88      0.89      0.89       282



KeyboardInterrupt: 